### Created by yunsuxiaozi

### Import necessary libraries

In [ ]:
#https://www.kaggle.com/code/ttahara/hms-hbac-resnet34d-baseline-training
#https://www.kaggle.com/code/ttahara/hms-hbac-resnet34d-baseline-inference
#necessary
import pandas as pd#导入csv文件的库
import numpy as np#进行矩阵运算的库
import torch #一个深度学习的库Pytorch
import torch.nn as nn#neural network,神经网络
import torch.nn.functional as F#神经网络函数库
import torchvision.transforms as transforms#Pytorch下面的图像处理库,用于对图像进行数据增强
#设置随机种子
import random
import warnings#避免一些可以忽略的报错
warnings.filterwarnings('ignore')#filterwarnings()方法是用于设置警告过滤器的方法，它可以控制警告信息的输出方式和级别。

### trained_model

In [ ]:
model = torch.load('/kaggle/input/hms-baseline-resnet34d-training/HMS_resnet.pth')

### Config

In [ ]:
class Config:
    seed=2024
    image_transform=transforms.Resize((512, 512))

### Seed

In [ ]:
def seed_everything(seed):
    torch.backends.cudnn.deterministic = True#将cuda加速的随机数生成器设为确定性模式
    torch.backends.cudnn.benchmark = True#关闭CuDNN框架的自动寻找最优卷积算法的功能，以避免不同的算法对结果产生影响
    torch.manual_seed(seed)#pytorch的随机种子
    np.random.seed(seed)#numpy的随机种子
    random.seed(seed)#python内置的随机种子
seed_everything(Config.seed)

### Import test dataset

In [ ]:
test_df=pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/test.csv")
submission=pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv")
submission=submission.merge(test_df,on='eeg_id',how='left')
submission['path']=submission['spectrogram_id'].apply(lambda x: "/kaggle/input/hms-harmful-brain-activity-classification/test_spectrograms/"+str(x)+".parquet" )
submission.head()

### Prediction

In [ ]:
paths=submission['path'].values
model.eval()
test_pred=[]
with torch.no_grad():
    for path in paths:
        eps=1e-6
        data=pd.read_parquet(path)
        #这里最小值是0,故用-1填充.第一列是时间列,故去掉 ,行是不同列,列是时间
        data = data.fillna(-1).values[:,1:].T
        #选取一段时间的数据进行训练
        data=data[:,0:300]#(400,300)
        data=np.clip(data,np.exp(-6),np.exp(10))#最大值为89209464.0
        data= np.log(data)#对数变换
        #对数据进行归一化
        data_mean=data.mean(axis=(0,1))
        data_std=data.std(axis=(0,1))
        data=(data-data_mean)/(data_std+eps)
        data_tensor = torch.unsqueeze(torch.Tensor(data), dim=0)
        data=Config.image_transform(data_tensor)
        pred=F.softmax(model(data.unsqueeze(0)))[0]
        pred=pred.detach().cpu().numpy()
        test_pred.append(pred)
test_pred=np.array(test_pred)
test_pred

### Submission

In [ ]:
submission=pd.read_csv("/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv")
labels=['seizure','lpd','gpd','lrda','grda','other']
for i in range(len(labels)):
    submission[f'{labels[i]}_vote']=test_pred[:,i]
submission.to_csv("submission.csv",index=None)
submission.head()